In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as plt
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler,StandardScaler
from imblearn.combine import SMOTEENN
sns.set(color_codes=True)
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/merchandise-popularity-prediction-challenge/Train.csv')
test = pd.read_csv('../input/merchandise-popularity-prediction-challenge/Test.csv')

In [ ]:
df.head()

In [ ]:
test.head()

In [ ]:
df.columns

In [ ]:
for i in test.columns:
    fig,ax = plt.subplots(nrows=1,ncols=2)
    sns.distplot(df[i],ax=ax[0],color="blue")
    sns.distplot(test[i],ax=ax[1],color="green")

In [ ]:
for i in df.columns:
    print('Unique Values in Column {} is {}'.format(i,len(df[i].unique())))

In [ ]:
sns.countplot(df.popularity)

In [ ]:
df.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
X = df.drop('popularity',axis=1)
Y = df.popularity

In [ ]:
rs = RobustScaler(
with_centering=True,
with_scaling=True,
quantile_range=(25.0, 75.0),
copy=True,
)
x = rs.fit_transform(X)
sc = StandardScaler()
X_ = sc.fit_transform(x)

In [ ]:
X = pd.DataFrame(X_,columns=X.columns)

In [ ]:
sns.distplot(X)

In [ ]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression , RidgeClassifier, Huber
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier , ExtraTreesClassifier , AdaBoostClassifier
from sklearn.model_selection import KFold ,StratifiedKFold,RepeatedKFold,RepeatedStratifiedKFold,RandomizedSearchCV
from sklearn.metrics import log_loss

In [ ]:
# smote_enn = SMOTEENN()
# X,Y = smote_enn.fit_resample(X,Y)

In [ ]:
from collections import Counter
Counter(Y)

In [ ]:
folds = StratifiedKFold(n_splits=10,shuffle=True ,random_state=42)
# folds = RepeatedStratifiedKFold(n_splits=5, n_repeats=2,random_state=42)

In [ ]:
oof_preds_xg = []
oof_score_xg = []
for i,(train_idx,test_idx) in enumerate(folds.split(X,Y)):
    train_set = (X.iloc[train_idx],Y.iloc[train_idx])
    test_set = (X.iloc[test_idx],Y.iloc[test_idx])
    model = XGBClassifier()
    model.fit(*train_set)
    preds = model.predict_proba(test_set[0])
    score = log_loss(test_set[1],preds)
    oof_pred_fold = model.predict_proba(sc.transform(rs.transform(test)))
    oof_preds_xg.append(oof_pred_fold)
    print('Score - ',score)
    oof_score_xg.append(score)
print("Mean OOF Score :",np.mean(oof_score_xg))
final_preds_xg = np.mean(np.array(oof_preds_xg),axis=0)

In [ ]:
oof_preds_cat = []
oof_score_cat = []
for i,(train_idx,test_idx) in enumerate(folds.split(X,Y)):
    train_set = (X.iloc[train_idx],Y.iloc[train_idx])
    test_set = (X.iloc[test_idx],Y.iloc[test_idx])
    model = CatBoostClassifier(verbose=0)
    model.fit(*train_set)
    preds = model.predict_proba(test_set[0])
    score = log_loss(test_set[1],preds)
    oof_pred_fold = model.predict_proba(sc.transform(rs.transform(test)))
    oof_preds_cat.append(oof_pred_fold)
    print('Score - ',score)
    oof_score_cat.append(score)
print("Mean OOF Score :",np.mean(oof_score_cat))
final_preds_cat = np.mean(np.array(oof_preds_cat),axis=0)

In [ ]:
oof_preds_lgb = []
oof_score_lgb = []
for i,(train_idx,test_idx) in enumerate(folds.split(X,Y)):
    train_set = (X.iloc[train_idx],Y.iloc[train_idx])
    test_set = (X.iloc[test_idx],Y.iloc[test_idx])
    model = LGBMClassifier(verbose=0)
    model.fit(*train_set)
    preds = model.predict_proba(test_set[0])
    score = log_loss(test_set[1],preds)
    oof_pred_fold = model.predict_proba(sc.transform(rs.transform(test)))
    oof_preds_lgb.append(oof_pred_fold)
    print('Score - ',score)
    oof_score_lgb.append(score)
print("Mean OOF Score :",np.mean(oof_score_lgb))
final_preds_lgb = np.mean(np.array(oof_preds_lgb),axis=0)

**HyperParameter Tuning of RandomForest Classifier**

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]



random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}




rf_random = RandomizedSearchCV(estimator=RandomForestClassifier(),param_distributions=random_grid,
                              n_iter=10,cv=5,verbose=2,random_state=42,n_jobs=1)

rf_random.fit(X,Y)

In [ ]:
rf_random.best_params_

In [ ]:
oof_preds_rf = []
oof_score_rf = []
for i,(train_idx,test_idx) in enumerate(folds.split(X,Y)):
    train_set = (X.iloc[train_idx],Y.iloc[train_idx])
    test_set = (X.iloc[test_idx],Y.iloc[test_idx])
    model = RandomForestClassifier(n_estimators= 1000,min_samples_split= 2,min_samples_leaf= 1,max_features= 'sqrt',max_depth= 25)
    model.fit(*train_set)
    preds = model.predict_proba(test_set[0])
    score = log_loss(test_set[1],preds)
    oof_pred_fold = model.predict_proba(sc.transform(rs.transform(test)))
    oof_preds_rf.append(oof_pred_fold)
    print('Score - ',score)
    oof_score_rf.append(score)
print("Mean OOF Score :",np.mean(oof_score_rf))
final_preds_rf = np.mean(np.array(oof_preds_rf),axis=0)

In [ ]:
submission = pd.DataFrame(final_preds_rf)
submission.to_csv("submission.csv",index=None)
submission.head()